In [144]:
import loglib 
from matplotlib import pyplot as plt
from collections import defaultdict
from math import *
import numpy as np
from pyquaternion import Quaternion

In [145]:
log = loglib.RTRKLog()
log.load_raw_pickle("./tripoli_2/dad_h550_orange_crash_GOOD.pickle")
log.calibrate("./tracker2_cfg.json")
sls = log.split_logs()
for i,sl in enumerate(sls):
    print(f"Sublog {i}:")
    for ev in sl.get_events():
        print(f"\t{ev}")

Sublog 0:
	Device boot @ T+3.40s
	Log rate 10Hz @ T+975.67s
	Log rate 200Hz @ T+1931.07s
	Liftoff @ T+1931.07s
	Log rate 60Hz @ T+1941.08s


In [146]:
cur_log = sls[0]
cur_log.normalize()
cur_log.sort()
for ev in cur_log.get_events():
    print(f"\t{ev}")

log_data = [f for f in cur_log.frames if isinstance(f.data, loglib.LogDataDefault)]

	Device boot @ T-1927.67s
	Log rate 10Hz @ T-955.40s
	Log rate 200Hz @ T-0.00s
	Liftoff @ T+0.00s
	Log rate 60Hz @ T+10.00s


In [147]:
times = [f.timestamp for f in log_data]
datas = [f.data for f in log_data]

In [148]:
def hPa2alt(hPa):
    return 44330.0 * (1 - pow(hPa / 1013.25, 0.190284));

In [149]:
def axesSwapVector(vec):
    return (
		-vec[0], 
		-vec[1],
		vec[2]
	)
    
def interpolateAccel(lsm_acc, adxl_acc, low=15.8, high=16.0):
    out = [0,0,0]
    for ax in range(3):
        lsm_v = lsm_acc[ax]
        adxl_v = adxl_acc[ax]
        
        avg = (lsm_v + adxl_v) * 0.5
        interp_t = (avg - low)/(high-low)
   
        # Clamp
        if (interp_t < 0):
            interp_t = 0.0
        elif (interp_t > 1.0):
            interp_t = 1.0
        
        out[ax] = (1.0-interp_t)*lsm_v + (interp_t)*adxl_v
        
    return out

In [150]:
# TODO: For the real tracker, set much more conservative thresholds for reliability...

class LandingDetector():
    def __init__(self) -> None:
        self.state = 0
        self.alt_last = 0.0;
    
    # TODO: Use filtered V-Speed, need a better way because there's something funky going on with acceleration...
    def calculate(self, acc_g: list, gyro_dps: list, dt: float):
        accmag = abs(sqrt(sum([a**2 for a in acc_g]))-1.0)
        gyrmag = sum([abs(g) for g in gyro_dps])
        # alt_delta = (filt_alt-self.alt_last)/dt
        # self.alt_last = filt_alt
        # print(alt_delta)
        # (abs(alt_delta) < 1.5) and
        
        # These are good values I think...
        if (gyrmag < 5.0) and (accmag < 0.25):
            self.state += dt
        else:
            self.state = 0
            
    def is_landed(self):
        # Inactive for at least 0.1 second NEED TO INCREASE???
        return self.state > 0.1
    
    

In [151]:
alts_raw = []
alts_filtered = []
accel_world = []
accel_world_adxl = []
vel_world = [(0,0,0)]
pos_world = [(0,0,0)]

launched = False
launched_idx = 0

dts = []

alpha = 0.15

hPa_filtered = 0
hPa_filter_init = False
alt_0 = None

land_det = LandingDetector()
landed_idx = -1

tl = times[0] + (times[0]-times[1])
for i, (t, data) in enumerate(zip(times, datas)):
    dt = t-tl
    tl = t
    dts.append(dt)
    
    if (hPa_filter_init):
        hPa_filtered = hPa_filtered*(1-alpha) + alpha*data.lps_press
    else:
        hPa_filtered = data.lps_press
        alt_0 = hPa2alt(data.lps_press)
        hPa_filter_init = True
        
    alt_m_raw = hPa2alt(data.lps_press)-alt_0
    alt_m_filt = hPa2alt(hPa_filtered)-alt_0
    
    # Acceleration switchover (interpolation)
    lsm_correct = axesSwapVector(data.lsm_acc)
    adxl_correct = axesSwapVector(data.adxl_acc)
    acc_interpolated = interpolateAccel(lsm_correct, adxl_correct)
    
    # acc = axesSwapVector(data.lsm_acc)
    quat = Quaternion(*data.orientation_quat)
    world_acc = quat.rotate(acc_interpolated)
    world_acc = (
        world_acc[0] * 9.81,
        world_acc[1] * 9.81,
        (world_acc[2]-1.0) * 9.81,
    )
    
    if (abs(world_acc[2]) > 30.0 and not launched):
        launched = True
        launched_idx = i
        print(f"Launched @ t={t}")
    
    if (launched):
        vel_world.append((
            (vel_world[-1][0] + world_acc[0]*dt),
            (vel_world[-1][1] + world_acc[1]*dt),
            (vel_world[-1][2] + world_acc[2]*dt),
        ))
        pos_world.append((
            (pos_world[-1][0] + vel_world[-1][0]*dt),
            (pos_world[-1][1] + vel_world[-1][1]*dt),
            (pos_world[-1][2] + vel_world[-1][2]*dt),
        ))
        
        land_det.calculate(data.lsm_acc, data.lsm_gyr, dt)
        if (land_det.is_landed() and landed_idx == -1):
            print(f"Landed @ t={t}")
            landed_idx = i
            
    else:
        vel_world.append(vel_world[-1])
        pos_world.append(pos_world[-1])
        
    

    alts_raw.append(alt_m_raw)
    alts_filtered.append(alt_m_filt)
    accel_world.append(world_acc)
    
del vel_world[0]
del pos_world[0]
        
%matplotlib qt
# plt.plot(times, alts_raw, label='raw altitude')
# plt.plot(times, alts_filtered, label='alpha=0.15')
# plt.plot(times, [d.filtered_altitude_m for d in datas], label='filtered altitude')
# plt.plot(times, [wacc[0] for wacc in accel_world], label='accel X')
# plt.plot(times, [wacc[1] for wacc in accel_world], label='accel Y')
# plt.plot(times, [wacc[2] for wacc in accel_world], label='accel Z')
# plt.plot(times, [v[2] for v in vel_world], label='Z velocity')

plt.plot(times[launched_idx:landed_idx], [a[2] for a in accel_world][launched_idx:landed_idx], label='world accel')
plt.plot(times[launched_idx:landed_idx], [v[2] for v in vel_world][launched_idx:landed_idx], label='world vel')
plt.plot(times[launched_idx:landed_idx], [p[2] for p in pos_world][launched_idx:landed_idx], label='world pos')
# plt.plot(times, alts_raw, label='raw altitude')
plt.plot(times[launched_idx:landed_idx], alts_filtered[launched_idx:landed_idx], label='baro altimetry w/ alpha=0.15')  


  
plt.legend()
plt.show()

Launched @ t=0.005014999999957581
Landed @ t=14.021848000000091


In [152]:
# ax = plt.figure().add_subplot(projection='3d')
# ax.plot(
#     *zip(*pos_world),
#     label='flight'
# )
# ax.legend()
# plt.show()
# plt.plot(times, [sqrt(sum([a**2 for a in d.lsm_acc])) for d in datas], label='accel magnitude')

plt.legend()
plt.show()

In [153]:
a = 0.15
f_sampling = 60
f_cutoff = (f_sampling / (2*pi))*acos(1-((a*a)/(2*(1-a))))

ALPHA = 0.15
r = 60/256
ALPHA*r

0.03515625